In [39]:
import pandas as pd
import numpy as np
import seaborn as sns

In [4]:
filepath = '/Users/jacquelinethibault/Desktop/ResponseApproximation/fa23urap/test3.txt'
df = pd.read_csv(filepath, delimiter = ',')

In [5]:
df.head()

,Unnamed: 0,Event ID,Energy,Theta,Phi,Scatter Angle,Path Length (cm),Theta Bin,Phi Bin,Scatter Angle Bin
0,0,1,509.8545,95.111090,-79.695154,22.103606,1.122497,96.630738,-74.931512,23.016826
1,1,2,510.7631,90.000000,-45.000000,40.497715,0.282843,90.000000,-45.000000,41.500863
2,2,3,509.1930,165.046589,33.690068,82.068576,2.794638,180.000000,43.264295,83.771117
3,3,4,511.8270,103.109018,-152.241459,63.178452,2.204541,103.240520,-137.726311,63.850514
4,4,6,510.1480,170.251968,-104.036243,68.882363,2.435159,180.000000,-103.348727,72.330605


In [23]:
meanitems=df.groupby('Phi Bin').count().iloc[:,0].mean()#.append(df.groupby('Phi Bin').count().iloc[9:,0]).mean()
display(meanitems - df.groupby('Phi Bin').count().iloc[:,0])
display(meanitems)

Phi Bin
-165.173520     14403.947368
-137.726311     14087.947368
-124.159695     14704.947368
-113.198591     14391.947368
-103.348727     14428.947368
-90.000000    -261889.052632
-74.931512      16987.947368
-64.536655      14236.947368
-55.007980      13796.947368
-45.000000       4896.947368
-23.198591      24621.947368
-0.000000     -172385.052632
 14.036243     199487.947368
 43.264295      16197.947368
 66.801409      14357.947368
 90.000000     -54140.052632
 120.963757     82742.947368
 148.671307     14659.947368
 179.530373     14408.947368
Name: Unnamed: 0, dtype: float64

288114.94736842107

In [34]:
df_dropped = df[df['Theta'] != 90][df['Phi'] != -90][df['Phi'] != 0][df['Phi']!=90]
meanitems=df_dropped.groupby('Phi Bin').count().iloc[:,0].mean()#.append(df.groupby('Phi Bin').count().iloc[9:,0]).mean()
display(meanitems - df_dropped.groupby('Phi Bin').count().iloc[:,0])

/var/folders/0p/p_3sbhs92f1cbn5q60vgrkxw0000gn/T/ipykernel_42030/2273803363.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_dropped = df[df['Theta'] != 90][df['Phi'] != -90][df['Phi'] != 0][df['Phi']!=90]


Phi Bin
-165.173520     13536.578947
-137.726311    -32680.421053
-124.159695    -18678.421053
-113.198591    -27911.421053
-103.348727    -32127.421053
-90.000000     -14577.421053
-74.931512     -35619.421053
-64.536655     -34072.421053
-55.007980     -27545.421053
-45.000000     -25646.421053
-23.198591     -22711.421053
-0.000000       72687.578947
 14.036243     139273.578947
 43.264295     -28445.421053
 66.801409     -13414.421053
 90.000000     101299.578947
 120.963757     34117.578947
 148.671307    -16900.421053
 179.530373    -30584.421053
Name: Unnamed: 0, dtype: float64

In [30]:
df.groupby(['Theta Bin', 'Phi Bin', 'Scatter Angle Bin']).count().head(50)

Unnamed: 0  Event ID  Energy  Theta  \
Theta Bin Phi Bin     Scatter Angle Bin                                        
27.791306 -165.173520 18.860194                 160       160     160    160   
                      23.016826                 162       162     162    162   
                      26.914189                 190       190     190    190   
                      30.642932                 255       255     255    255   
                      34.244176                 320       320     320    320   
                      37.816813                 411       411     411    411   
                      41.500863                 549       549     549    549   
                      45.304219                 587       587     587    587   
                      49.338713                 729       729     729    729   
                      53.739245                 578       578     578    578   
                      58.441006                 397       397     397    397   
                      63.850514                 371       371     371    371   
                      72.330605                 436       436     436    436   
                      83.771117                 569       569     569    569   
                      94.509979                1809      1809    1809   1809   
                      103.501217               6973      6973    6973   6973   
                      113.828575               5429      5429    5429   5429   
                      125.754105               1498      1498    1498   1498   
                      140.424144                456       456     456    456   
                      179.918674                196       196     196    196   
          -137.726311 18.860194                  66        66      66     66   
                      23.016826                  84        84      84     84   
                      26.914189                  72        72      72     72   
                      30.642932                 116       116     116    116   
                      34.244176                 153       153     153    153   
                      37.816813                 212       212     212    212   
                      41.500863                 254       254     254    254   
                      45.304219                 311       311     311    311   
                      49.338713                 301       301     301    301   
                      53.739245                 231       231     231    231   
                      58.441006                 192       192     192    192   
                      63.850514                 191       191     191    191   
                      72.330605                 258       258     258    258   
                      83.771117                 434       434     434    434   
                      94.509979                3648      3648    3648   3648   
                      103.501217               3601      3601    3601   3601   
                      113.828575                595       595     595    595   
                      125.754105                145       145     145    145   
                      140.424144                 71        71      71     71   
                      179.918674                 25        25      25     25   
          -124.159695 18.860194                  70        70      70     70   
                      23.016826                 102       102     102    102   
                      26.914189                 113       113     113    113   
                      30.642932                 138       138     138    138   
                      34.244176                 211       211     211    211   
                      37.816813                 248       248     248    248   
                      41.500863                 240       240     240    240   
                      45.304219                 272       272     272    272   
                      49.338713          

In [38]:
justbins_df=df[['Theta Bin', 'Phi Bin', 'Scatter Angle Bin']]
justbins_df

,Theta Bin,Phi Bin,Scatter Angle Bin
0,96.630738,-74.931512,23.016826
1,90.000000,-45.000000,41.500863
2,180.000000,43.264295,83.771117
3,103.240520,-137.726311,63.850514
4,180.000000,-103.348727,72.330605
...,...,...,...
5474179,61.094381,-45.000000,58.441006
5474180,61.094381,-124.159695,103.501217
5474181,90.000000,-55.007980,23.016826
5474182,88.538861,-45.000000,41.500863


In [ ]:
justbins_df[just]